# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [2]:
import numpy as np

In [3]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [4]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)


(9.99064257773601, 10.0)

In [10]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10**6):
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    
    before = np.exp(-rf_rate*mat)*np.ones_like(default_time[default_time >= mat])
    after = np.exp(-rf_rate*default_time[default_time < mat])*recovery
    
    return np.hstack([before,after]).mean()

# Call your function
print(corp_bond(mat, def_rate, rf_rate, recovery, n_sample))

# Find the mean and std by calling the function 100 times. 
std = [corp_bond(mat, def_rate, rf_rate, recovery, n_sample) for i in range(100)]
print(np.mean(std))
    



0.44412669061288207
0.44053188400690074


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [6]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=n_sample//2)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [8]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10**4, antithetic=True, mean_match=True):
    if antithetic and mean_match:
        U = np.random.uniform(size=n_sample//2)
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))        
        default_time += 1/def_rate-default_time.mean()    
    
    elif(antithetic):
        U = np.random.uniform(size=n_sample//2)
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        
    elif(mean_match):
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(U)
        default_time += 1/def_rate-default_time.mean()
        
    else:
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(U)
    
    before = np.exp(-rf_rate*mat)*np.ones_like(default_time[default_time >= mat])
    after = np.exp(-rf_rate*default_time[default_time < mat])*recovery
    
    return np.hstack([before,after]).mean()


# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both

method_1 = np.array([corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample=10**4, antithetic=True, mean_match=False) for i in range(100)])
method_2 = np.array([corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample=10**4, antithetic=False, mean_match=True) for i in range(100)])
method_3 = np.array([corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample=10**4, antithetic=True, mean_match=True) for i in range(100)])

method = [method_1,method_2,method_3]

for i in range(len(method)):
    print("method_%s: mean = %0.6f, std = %0.6f"%(i+1,method[i].mean(),method[i].std()))

method_1: mean = 0.440450, std = 0.001659
method_2: mean = 0.440270, std = 0.001668
method_3: mean = 0.440682, std = 0.001462


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [9]:
### Put the analytic expression for the corporate bond price
def corp_bond_analytic(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3):
    return def_rate/(def_rate+rf_rate)*(1-np.exp(-mat*(def_rate+rf_rate)))*recovery+np.exp(-mat*(def_rate+rf_rate))

print("analytic value is %0.6f"%corp_bond_analytic(mat, def_rate, rf_rate, recovery))
for i in range(len(method)):
    print("method_%s deviation: %0.6f"%(i+1,method[i].mean()-corp_bond_analytic(mat, def_rate, rf_rate, recovery)))

analytic value is 0.440409
method_1 deviation: 0.000041
method_2 deviation: -0.000139
method_3 deviation: 0.000273
